In [3]:
import pymongo
from mLabsignin import username, password
import pandas as pd
import string

#establish connection to db
conn = 'mongodb://%s:%s@ds149905.mlab.com:49905/heroku_236cmwk4' % (username, password)
client = pymongo.MongoClient(conn)

#url = "http://sn4hr.org/page/" + num + "/?s=school"

#define db
db = client.heroku_236cmwk4

#define collection
collection = db.school_attacks


In [4]:
cursor = collection.find()
results = list(cursor)

In [5]:
title_text = []
title_text_lower = []
body_text = []
body_text_lower = []
pub_date = []
article_link = []

for result in results:
    title_text.append(result["title_lowercase"])
    body_text.append(result["article_text_lower"])
    pub_date.append(result["date_published"])
    article_link.append(result["article_link"])

df = pd.DataFrame({
    "title": title_text,
    "body_text": body_text,
    "pub_date": pub_date,
    "article_link": article_link
})

In [6]:
df.head()

,article_link,body_text,pub_date,title
0,http://sn4hr.org/blog/2017/10/31/syrian-regime...,\nsnhr: syrian regime artillery fired shells i...,"October 31, 2017",syrian regime forces shelled mohammad naser as...
1,http://sn4hr.org/blog/2017/10/25/syrian-regime...,\nsnhr: syrian regime warplanes fired several ...,"October 25, 2017",syrian regime forces shelled school in al bouk...
2,http://sn4hr.org/blog/2017/10/18/syrian-regime...,\nsnhr: syrian regime warplanes fired missiles...,"October 18, 2017",syrian regime forces shelled hamoud al dabet s...
3,http://sn4hr.org/blog/2017/10/15/suspected-rus...,\nsnhr: suspected russian warplanes fired miss...,"October 15, 2017",suspected russian forces shelled tahtaya prima...
4,http://sn4hr.org/blog/2017/10/07/syrian-regime...,\nsnhr: syrian regime warplanes fired missiles...,"October 7, 2017",syrian regime forces shelled al hamzat school ...


In [12]:
#create body_text that is cleaned of non letters
letter_list = list(string.ascii_letters)
letter_list.append(" ")

df['clean_text'] = ""

for index, row in df.iterrows():
    text = row['body_text']
    for letter in text:
        if letter not in letter_list:
            text = text.replace(letter, "")
    df.set_value(index, 'clean_text', text)

df.head()


snhr syrian regime artillery fired shells in front of mohammad naser ashoush primary school in jisreen town in eastern ghouta in damascus suburbs governorate causing a massacre in addition to damaging the school building and furniture partially on october  
snhr syrian regime warplanes fired several missiles near al baath school in al boukamal city in deir ezzour governorate eastern suburbs damaging it on october  
snhr syrian regime warplanes fired missiles near hamoud al dabet school in gharaneej city in deir ezzour governorate eastern suburbs damaging it october  
snhr suspected russian warplanes fired missiles on tahtaya primary school in al shamali neighborhood in tahtaya village in idlib governorate southern suburbs severely damaging its building and fence damaging its furniture and rendering it inoperable on october  
snhr syrian regime warplanes fired missiles near al hamzat school in gharbiya village in deir ezzour governorate eastern suburbs partially damaging it on october  

In [13]:
# create a dictionary of words to inspect for most used words relating to damage
words_dict = {}

for index, row in df.iterrows():
    print(row['clean_text'])
    art_text_list = row['clean_text'].split(" ")
    for word in art_text_list:
        if word in words_dict.keys():
            words_dict[word] += 1
        else:
            words_dict[word] = 1

snhr syrian regime artillery fired shells in front of mohammad naser ashoush primary school in jisreen town in eastern ghouta in damascus suburbs governorate causing a massacre in addition to damaging the school building and furniture partially on october  
snhr syrian regime warplanes fired several missiles near al baath school in al boukamal city in deir ezzour governorate eastern suburbs damaging it on october  
snhr syrian regime warplanes fired missiles near hamoud al dabet school in gharaneej city in deir ezzour governorate eastern suburbs damaging it october  
snhr suspected russian warplanes fired missiles on tahtaya primary school in al shamali neighborhood in tahtaya village in idlib governorate southern suburbs severely damaging its building and fence damaging its furniture and rendering it inoperable on october  
snhr syrian regime warplanes fired missiles near al hamzat school in gharbiya village in deir ezzour governorate eastern suburbs partially damaging it on october  

In [15]:
#create dataframe of word counts
words_df = pd.DataFrame([words_dict])
words_df = words_df.transpose()
words_df = words_df.sort_values(0, ascending = False)

 688
in 632
school 363
al 314
snhr 294
on 288
fired 271
damaging 255
warplanes 234
governorate 203
missiles 199
and 174
it 166
city 152
its 142
partially 133
suburbs 126
idlib 122
the 120
a 120
building 108
government 108
regime 79
town 77
russian 68
aleppo 60
severely 59
of 59
near 59
fence 58
suspected 58
neighborhood 53
rendering 48
inoperable 48
village 48
november 45
missile 44
primary 42
southern 41
syrian 40
october 38
deir 36
high 33
destroying 32
september 32
furniture 32
artillery 32
ezzour 31
shells 31
western 28
raqqa 28
december 27
august 27
eastern 26
by 26
march 26
causing 25
from 25
june 23
april 23
for 22
international 22
may 22
northern 22
coalition 22
damascus 21
to 20
dropped 19
february 19
july 19
kafr 17
jisr 17
died 17
isis 16
mortar 16
barrel 16
children 16
maaret 16
girls 16
at 16
homs 16
january 15
forces 15
schools 15
massacre 14
daraa 14
opposition 13
least 13
aviation 13
hama 13
martyr 13
armed 13
local 12
killing 12
bomb 11
boys 10
elementary 10
controlled

In [60]:
### found words 
# partially
# severely
# damaging 
# damaged
# completely
# inoperable
level_of_damage = ['severely damaging', 
                   'severely damaged',
                   'severely', 
                   'completely damaging', 
                   'completely damaged', 
                   'completely', 
                   'partially damagaged',
                   'partially damaging',
                   'partially',
                   'rendering inoperable',
                   'destroying most',
                   'destroying part',
                   'inoperable',
                   'damage',
                   'damaging',
                   'destroying'
                  ]
#attackers
#syrian
#russian

#deaths
# massacre
# killing
# killed
# died

In [61]:
df['damage_level'] = ''

for index, row in df.iterrows():
    text = row['clean_text']
    for word in level_of_damage:
        if word in text:
            df.set_value(index, 'damage_level', word)
            break
    

In [62]:
df['damage_level'].value_counts()

partially damaging    77
damaging              62
partially             43
severely              43
                      40
severely damaging     16
damage                11
inoperable            10
completely             7
destroying part        4
destroying most        1
Name: damage_level, dtype: int64

In [64]:
# verify partially ranking
partially = df[df['damage_level'] == 'destroying']
for index, row in partially.iterrows():
    print(row['body_text'])
    

In [85]:
df['school'] = ""

stop_list = ["on",
            "near",
            "targeted",
            "inside"]

for index, row in df.iterrows():
    text = row['clean_text']
    text_list = text.split(" ")
    try:
        school_index = text_list.index('school')
    except: 
        try: 
            school_index = text_list.index('schools')
        except:
            break
    prior_text = text_list[: school_index + 1]
    for word in stop_list: 
        if word in prior_text: 
            school = prior_text[prior_text.index(word) + 1 :]
            school = " ".join(school)
            break
        else: 
            school = "no school found"
    print('----------')
    print(text)
    print("school:" + school)
    df.set_value(index, 'school', school)
        

            
        
    
            

----------
snhr syrian regime artillery fired shells in front of mohammad naser ashoush primary school in jisreen town in eastern ghouta in damascus suburbs governorate causing a massacre in addition to damaging the school building and furniture partially on october  
school:no school found
----------
snhr syrian regime warplanes fired several missiles near al baath school in al boukamal city in deir ezzour governorate eastern suburbs damaging it on october  
school:al baath school
----------
snhr syrian regime warplanes fired missiles near hamoud al dabet school in gharaneej city in deir ezzour governorate eastern suburbs damaging it october  
school:hamoud al dabet school
----------
snhr suspected russian warplanes fired missiles on tahtaya primary school in al shamali neighborhood in tahtaya village in idlib governorate southern suburbs severely damaging its building and fence damaging its furniture and rendering it inoperable on october  
school:tahtaya primary school
----------
sn

In [99]:
df.head()

,article_link,body_text,pub_date,title,clean_text,damage_level,school
0,http://sn4hr.org/blog/2017/10/31/syrian-regime...,\nsnhr: syrian regime artillery fired shells i...,"October 31, 2017",syrian regime forces shelled mohammad naser as...,snhr syrian regime artillery fired shells in f...,partially,no school found
1,http://sn4hr.org/blog/2017/10/25/syrian-regime...,\nsnhr: syrian regime warplanes fired several ...,"October 25, 2017",syrian regime forces shelled school in al bouk...,snhr syrian regime warplanes fired several mis...,damaging,al baath school
2,http://sn4hr.org/blog/2017/10/18/syrian-regime...,\nsnhr: syrian regime warplanes fired missiles...,"October 18, 2017",syrian regime forces shelled hamoud al dabet s...,snhr syrian regime warplanes fired missiles ne...,damaging,hamoud al dabet school
3,http://sn4hr.org/blog/2017/10/15/suspected-rus...,\nsnhr: suspected russian warplanes fired miss...,"October 15, 2017",suspected russian forces shelled tahtaya prima...,snhr suspected russian warplanes fired missile...,severely damaging,tahtaya primary school
4,http://sn4hr.org/blog/2017/10/07/syrian-regime...,\nsnhr: syrian regime warplanes fired missiles...,"October 7, 2017",syrian regime forces shelled al hamzat school ...,snhr syrian regime warplanes fired missiles ne...,partially damaging,al hamzat school


In [100]:
df.to_csv('bombings_with_schools.csv', index = False)

In [87]:
df['school'].value_counts()

a school                                                                              28
no school found                                                                        9
school                                                                                 8
schools                                                                                5
primary school                                                                         5
high school                                                                            4
walid shaaban high school                                                              4
al wataniyeh school                                                                    3
the industrial high school                                                             3
abi al alaa preparatory school                                                         3
the secondary school                                                                   3
al senaa school      

In [90]:
no_schools_df = df[(df['school'] == 'no school found') | 
                   (df['school'] == ' a school') | 
                   (df['school'] == 'school') |
                   (df['school'] == 'schools') |
                   (df['school'] == 'high school') |
                   (df['school'] == 'primary school') |
                   (df['school'] == 'a primary school') |
                   (df['school'] == 'secondary school') |
                   (df['school'] == 'a secondary school') |
                   (df['school'] == ' a high school')]
len(no_schools_df)

34

In [96]:
for index, row in no_schools_df.iterrows():
    print(row["clean_text"])
    print("---------")

snhr syrian regime artillery fired shells in front of mohammad naser ashoush primary school in jisreen town in eastern ghouta in damascus suburbs governorate causing a massacre in addition to damaging the school building and furniture partially on october  
---------
snhr international coalition forces warplanes shelled fayez mansour school in al boukamal city in deir ezzour governorate eastern suburbs damaging it completely on august  
---------
snhr international coalition warplanes missiles fired on high school for girls in al sabha village in deir ezzour governorate eastern suburbs damaging it severely rendering it inoperable july  
---------
snhr syrian regime warplanes fired missiles on school sheltering idps in tafas city in daraa governorate western suburbs causing a massacre and damaging the school june  
---------
snhr international coalition warplanes fired missiles on primary school in al abbara village in raqqa governorate northern suburbs damaging it severely april  
----